In [1]:
import tensorflow as tf
import numpy as np
#from utils import build_model
import pandas as pd
from tqdm import tqdm
import imageio
import matplotlib.pyplot as plt
import keras
from keras.layers import Conv2D, Dropout, Flatten, Dense
from keras.regularizers import l2
import statsmodels.api as sm
from utils import build_model

In [7]:
checkpoint_path = '../../../../data/models/mc_dropout_cil/export/'

In [8]:
model = build_model()
model.load_weights(tf.train.latest_checkpoint(checkpoint_path))

In [9]:
# all training images and paths
path_all_imgs = '../../../../commaai_code/02_write_shards_optional/cil_shards/df_paths.csv'
all_img_df = pd.read_csv(path_all_imgs)
img_path_base = '../../../../data/commaai/test_files/val_files_unfiltered/'
density_path= '../../../../data/commaai/density/gaussian_density.csv'
density = pd.read_csv(density_path)
all_img_df = all_img_df[np.abs(all_img_df['true_y']) < 40].reset_index()

In [ ]:
# extract Bzetas by reading in images and predicting
labels = []
preds = []
for i in tqdm(range(0,all_img_df.shape[0])): 
    img = imageio.imread(str(img_path_base + all_img_df.loc[i,'path']))[:,:,0:3]/255
    pred = model.predict(img.reshape(1,66,200,3))
    label = all_img_df.loc[i,'true_y']
    labels.append(label)       
    preds.append(pred)

# rearrange to arrays
labels = np.array(labels)
preds = np.array(preds)

 57%|█████▋    | 54270/94570 [4:30:43<5:10:10,  2.17it/s] 

In [ ]:
np.save('preds_cil.npy', preds)
np.save('labels_cil.npy', labels)

In [ ]:
np.mean((labels - preds.reshape(10472,))**2)

In [ ]:
np.mean(np.abs((labels - preds.reshape(10472,))))

In [ ]:
np.mean([(labels - preds.reshape(10472,)) <= 6])

In [ ]:
np.mean([(labels - preds.reshape(10472,)) <= 2])

In [ ]:
plt.scatter(labels, preds.reshape(10472,), alpha = 0.1)
plt.plot([-40,40], [-40,40], color = 'red', alpha = 0.2)

## MC-Dropout

In [5]:
#preds_samp = np.load('mc_preds.npy')
preds = np.load('preds_cil.npy')
labels = np.load('labels_cil.npy')

FileNotFoundError: [Errno 2] No such file or directory: 'preds_cil.npy'

In [2]:
preds_1 = np.append(np.load('../../../../data/commaai/predictions/mc_dropout/mc_preds_cil_1.npy'), 
                  np.load('../../../../data/commaai/predictions/mc_dropout/mc_preds_cil_2.npy'), axis = 0)
preds_mc = np.mean(preds_1, axis = 1)

In [3]:
print("Performance of models without and with uncertainty" +
     "\nMSE:" +
      #"\nwithout uncertainty: " + str(np.mean((labels - preds.reshape(10472,))**2)) +
      "\nwith uncertainty: " + str(np.mean((labels - preds_mc.reshape(10472,))**2)) +
      "\nMAE:" +
      #"\nwithout uncertainty: " + str(np.mean(np.abs(labels - preds.reshape(10472,)))) +
      "\nwith uncertainty: " + str(np.mean(np.abs(labels - preds_mc.reshape(10472,)))) +
      "\nAccuracy :" +
      #"\nwithout uncertainty: " + str(np.mean([np.abs((labels - preds.reshape(10472,))) <= 6])) +
      "\nwith uncertainty: " + str(np.mean([np.abs((labels - preds_mc.reshape(10472,))) <= 6])) +
      "\nAccuracy II:" +
      #"\nwithout uncertainty: " + str(np.mean([np.abs((labels - preds.reshape(10472,))) <= 2])) +
      "\nwith uncertainty: " + str(np.mean([(labels - preds_mc.reshape(10472,)) <= 2])) 
     )

NameError: name 'labels' is not defined